In [ ]:
from __future__ import print_function
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import *
import pandas as pd
import matplotlib.pyplot as plt

# read positivity file
positive_words = []

with open("positive-words.txt") as f:
    positive_words = f.read().split()[213:] 

def calculate_positivity(lst):
    count = 0
    for word in lst:
        if word in positive_words:
            count = count + 1
    return count

    
spark = SparkSession.builder.appName("Yelp Analysis").config("spark.some.config.option", "some-value").getOrCreate()
df = spark.read.json("file:///Users/yashparikh/Project/final_five_star_review.json")

df.createOrReplaceTempView("fiveStar")
sqlDF = spark.sql("SELECT year(date) as year, month(date) as month, text FROM fiveStar month ORDER BY year, month").toPandas()
totalReviews = sqlDF.groupby(['year', 'month']).apply(lambda x: "%s" % ' '.join(x['text']))
totalReviews = totalReviews.to_frame().rename(columns= {0: 'all_reviews'})

year_and_month = totalReviews.index.values.tolist()

total_count = []
positive_count = []

for date in year_and_month:
	entire_text = totalReviews.loc[(date)].values.tolist()[0].split(" ")
	length = len(entire_text)
	total_count.append(length)
	
	pos_count = (calculate_positivity(entire_text)/float(length)) * 100
	positive_count.append(pos_count)

totalReviews["pos_count"] = positive_count
totalReviews.plot.line(x=totalReviews.index, y='pos_count', figsize=(15,10))
# print(totalReviews.loc[(2006,3)].values)